<a href="https://colab.research.google.com/github/khorneflakes-dev/Proyecto-Final-YELP/blob/main/ETL_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#descomprime el zip con los datasets de yelp
#from zipfile import ZipFile 
#with ZipFile('drive/MyDrive/Copia de Dataset Yelp.zip', 'r') as zipObj:
#  zipObj.extractall('drive/MyDrive/yelp')

In [1]:
import pandas as pd
pd.options.display.max_columns = None # para mostrar todas las columnas de los dataframes
import numpy as np


## obtaining integer user codes csv. ¡suprimible!

In [ ]:
users = pd.read_json("drive/MyDrive/yelp/Dataset Yelp/user.json", lines=True , chunksize=200000) #lee user.json
cods_users = pd.concat(users).user_id
cods_users = cods_users.reset_index()
cods_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   index    int64 
 1   user_id  object
dtypes: int64(1), object(1)
memory usage: 30.3+ MB


In [ ]:
cods_users.user_id.duplicated().sum()

0

In [ ]:
cods_users = cods_users.rename(columns={'index':'n_user_id'})

In [ ]:
cods_users.n_user_id.duplicated().sum()

0

In [ ]:
cods_users.to_csv('drive/MyDrive/yelp/codes_users.csv', index=False)

In [ ]:
del cods_users

## user.json ETL

In [ ]:
users = pd.read_json("drive/MyDrive/yelp/Dataset Yelp/user.json", lines=True , chunksize=400000) # definition of the iterator reader object

In [ ]:
col_to_drop=['name','elite','compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain',
'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos'] # droppable columns list
data_types={'average_stars':'uint8' , 'fans':'uint16', 'review_count':'uint16', 'cool':'uint32', 'useful':'uint32', 'funny':'uint32'} # data types dict for some columns

chunks=[] # define empty list for chunks
for chunk in users:  #iterate over the reader object users
  iter_chunk = chunk.drop(columns=col_to_drop).astype(data_types) # drop columns listed in col_to_drop from every chunk and change the data types according to data_types dict
  chunks.append(iter_chunk)  # append modified chunks into the chunks list
  
  


In [ ]:
users = pd.concat(chunks) # concatenate all the chunks in the chunks list to create users dataframe

In [ ]:
users.yelping_since = pd.to_datetime(users.yelping_since, format='%Y-%m-%d %H:%M:%S') # change column 'yelping_since' to datetime

In [ ]:
# intento de ahorro de memoria 
del chunks
del chunk
del iter_chunk

In [ ]:
users = users.reset_index() # create a new column with integers from the index values, this new columns is named automatically as 'index'

In [ ]:
users = users.rename(columns={'index':'n_user_id'}) # rename the recently created column of integers as n_user_id
users = users.astype({'n_user_id':'uint32'}) # change the type of column n_user_id to uint32

In [ ]:
# exportar tabla user transformada en formato parquet !!!!Suprimible!!!!
# users.drop(columns='user_id').to_parquet('drive/MyDrive/yelp/users.gzip', compression ='gzip',  index=False)

## review.json chunk load to dataframe reviews

In [ ]:
reviews = pd.read_json("drive/MyDrive/yelp/Dataset Yelp/review.json", lines=True , chunksize=200000)

In [ ]:
data_types={'stars':'uint8' , 'useful':'int16', 'funny':'int16', 'cool':'int16'}
chunks=[] # define empty list for chunks
for chunk in reviews:  #iterate over the reader object reviews
  chunks.append(chunk.astype(data_types)) # change the data types from every chunk according to data_types dict an append to chunks list


In [ ]:
reviews = pd.concat(chunks, ignore_index=True)

In [ ]:
reviews = reviews.reset_index(drop=True)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        uint8         
 4   useful       int16         
 5   funny        int16         
 6   cool         int16         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int16(3), object(4), uint8(1)
memory usage: 313.3+ MB


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   n_user_id      uint32        
 1   user_id        object        
 2   review_count   uint16        
 3   yelping_since  datetime64[ns]
 4   useful         uint32        
 5   funny          uint32        
 6   cool           uint32        
 7   friends        object        
 8   fans           uint16        
 9   average_stars  uint8         
dtypes: datetime64[ns](1), object(2), uint16(2), uint32(4), uint8(1)
memory usage: 85.3+ MB


In [ ]:
del chunks
del chunk

In [ ]:
# exportar tabla user transformada en formato parquet
#reviews.drop(columns='user_id').to_parquet('drive/MyDrive/yelp/reviews.gzip', compression ='gzip',  index=False)

## change of alphanumerics ids for integer ids : it's found 33 reviews without user_id

In [ ]:
reviews.user_id.unique().shape

(1987929,)

In [ ]:
reviews = reviews.merge(users[['user_id','n_user_id']] , how='inner' , on='user_id') # inserta la columna n_user_id en la tabla reviews


In [ ]:
len(reviews[reviews.n_user_id.isnull()]) # busca reviews de ususarios que no estan en la tabla de users

0

In [ ]:
import gc
gc.collect()

## Crear una base de datos con sqlite3

In [ ]:
# CREATING THE TABLE
import sqlite3

conn = sqlite3.connect('drive/MyDrive/yelp/PFDB.db')
print("Opened database successfully");


reviews.to_sql('reviews', conn, if_exists='replace')

conn.execute(
    """
    create table reviews as 
    select * from reviews
    """)

print("Table created successfully");

conn.close()

Opened database successfully


# pyspark

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 47.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=538b22f7ad771a42a95dfe029f49af71e8f289aeed71224c52440996392cc70d
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]")\
        .appName("colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "5g")\
        .getOrCreate()
# Check Spark Session Information
spark

In [4]:
df = spark.read.json("drive/MyDrive/yelp/Dataset Yelp/user.json")

In [5]:
df.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [ ]:
df.show(5)

In [ ]:
df.count()

1987897

In [7]:
df_pd = df.toPandas()

Py4JJavaError: ignored